In [ ]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 61kB 23.1MB/s 
     |████████████████████████████████| 471kB 41.7MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4536331 sha256=640a60c1d38a898c3dc2650118a3a991f3153b721dc196a56c559c34e12065f9
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2019.1.1-py2.py3-none-any.whl size=58424 sha256=c48f5b9e915df2b6297c8e2869351c6097e8193f8df431ac401582d5416d4360
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Created wheel for mako: filename=Mako-1.1.0-cp36-none-any.whl size=75363 sha256=8c87d6f1e361799c7990ce48edbf8254484c6d2ef40c197d04221c0f8edf0b34
  Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Successfully built pycuda pytools mako


In [ ]:
import math
import numpy as np
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule
import pycuda.driver as drv
import pycuda.autoinit

In [ ]:
module = SourceModule("""
texture<int, 2, cudaReadModeElementType> tex_2d;
__global__ void read_texture_2d(int nx, int ny, int *res){
    int x = threadIdx.x + blockDim.x * blockIdx.x;
    int y = threadIdx.y + blockDim.y * blockIdx.y;
    int ij = nx * y + x;
    if (x < nx && y < ny){
        res[ij] = tex2D(tex_2d, x, y);
    }
}
""")

In [ ]:
read_tex_2d = module.get_function("read_texture_2d")

In [ ]:
num_components = np.int32(20)
arr = np.arange(num_components, dtype=np.int32)

In [ ]:
np.random.shuffle(arr)

In [ ]:
num_x, num_y = np.int32(5), np.int32(4)
arr = arr.reshape(num_y, num_x)

In [ ]:
arr

array([[ 2,  4, 16,  3, 11],
       [ 6,  1, 18, 17, 14],
       [ 8,  0,  5, 19, 12],
       [ 7,  9, 13, 15, 10]], dtype=int32)

In [ ]:
arr_gpu = gpuarray.to_gpu(arr)
res_gpu = gpuarray.zeros([num_y, num_x], dtype=np.int32)

In [ ]:
tex_2d = module.get_texref("tex_2d")

In [ ]:
drv.matrix_to_texref(arr, tex_2d, order="C")

In [ ]:
threads_per_block = (16, 16, 1)
block_x = math.ceil(num_x / threads_per_block[0])
block_y = math.ceil(num_y / threads_per_block[1])
blocks_per_grid = (block_x, block_y, 1)

In [ ]:
read_tex_2d(num_x, num_y, res_gpu, block=threads_per_block, grid=blocks_per_grid, texrefs=[tex_2d])

In [ ]:
res_gpu.get()

array([[ 2,  4, 16,  3, 11],
       [ 6,  1, 18, 17, 14],
       [ 8,  0,  5, 19, 12],
       [ 7,  9, 13, 15, 10]], dtype=int32)